## Importação das Bibiliotecas

In [3]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

In [4]:
# Crie um SparkSession
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


RuntimeError: Java gateway process exited before sending its port number

## Processo de Leitura do CSV de Participantes


In [3]:
arqEnem = 'E:\Estudos\SQL\Datasets\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'

dfDadosGerais = spark.read.csv(arqEnem, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')


In [4]:
dfParticipantes = dfDadosGerais.select(dfDadosGerais.columns[:12])

In [5]:
dfPreencher = dfParticipantes.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER

In [6]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "Participantes"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [7]:
# Nome da nova tabela que será criada
nomeTabela = 'Participantes'

In [8]:
#Conectando via jdbc
#No método option, foi necessário passar os argumentos ssl e trustservercertificate como true, por se tratar de um banco local sem SSL.
try:
  jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()
  
  jdbcDF.show(40)
  print(jdbcDF.count())
except ValueError as error :
    print("Connector write failed", error)


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+
|210057943671|  2022|             14|      M|              2|          2|               1|              1|              2|        1|       -2|           0|
|210057516120|  2022|             14|      M|              2|          1|               1|              1|             16|        1|       -2|           0|
|210055432769|  2022|             10|      F|              1|          1|               1|              1|              8|        1|       -2|           0|
|210055590133|  2022|              3|      F|              1|   

In [9]:
# Configurações do banco de dados
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = "DROP TABLE dbo.Participantes"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [10]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [11]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela Participantes criada com sucesso no banco de dados.
